<a href="https://colab.research.google.com/github/satyabratkumarsingh/option-portfolio-encoder-decoder/blob/main/Train_Set_Transformer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install torch
!pip install comet_ml
!pip install tqdm
!pip install matplotlib

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import os
def delete_file_from_drive(full_file_path):
  if os.path.exists(full_file_path):
      try:
          os.remove(full_file_path)
          print(f"File '{full_file_path}' successfully deleted from Google Drive.")
      except Exception as e:
          print(f"Error deleting file '{full_file_path}': {e}")
  else:
      print(f"File '{full_file_path}' not found at '{full_file_path}'.")


In [1]:
import random
import numpy as np
import torch
import itertools
from itertools import product
from torch.utils.data import Dataset, DataLoader
import gc # For garbage collection
import numpy as np
import yfinance as yf


# Parameters
MU = 0.05
T = 1.0 # Time to maturity
NOISE_STD = 0.005
MIN_PRICE_RANGE = 100
MAX_PRICE_RANGE = 500

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def calibrate_sigma_from_sp500(period="1y", ticker="SPY"):
    """
    Calibrate volatility (sigma) using S&P500 proxy (SPY ETF).
    Computes realized annualized volatility from historical data.

    Args:
        period (str): Period to download (e.g. "1y", "2y").
        ticker (str): Symbol to use, default SPY (ETF for S&P 500).

    Returns:
        float: Annualized volatility (sigma).
    """
    # Download daily adjusted close prices
    data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)

    closes = data["Close"].dropna()

    # Compute log returns
    log_returns = np.log(closes / closes.shift(1)).dropna()

    # Daily volatility
    sigma_daily = log_returns.std()

    # Annualized volatility
    sigma_annual = sigma_daily * np.sqrt(252)

    # Convert safely to float
    return sigma_annual.item() if hasattr(sigma_annual, "item") else float(sigma_annual)

# Example: update SIGMA
SIGMA = calibrate_sigma_from_sp500()
print(f"Calibrated SP500 1Y Volatility (Sigma): {SIGMA:.4f}")


[*********************100%***********************]  1 of 1 completed

Calibrated SP500 1Y Volatility (Sigma): 0.1932


In [2]:

TFR_RATIO = 0.5

def generate_hybrid_strikes(S_0, option_types, rng, training_friendly_ratio=TFR_RATIO):
    n = len(option_types)
    K_prices = np.zeros(n, dtype=np.float32)

    for i in range(n):
        if rng.random() < training_friendly_ratio:  # 40% chance
            K_prices[i] = S_0 * rng.uniform(0.90, 1.20)  # Mixed ITM/OTM
        else:
            # Realistic strikes
            if option_types[i] == "call":
                K_prices[i] = S_0 * rng.uniform(0.95, 1.15)  # Mix of ITM/OTM calls
            else:
                K_prices[i] = S_0 * rng.uniform(0.85, 1.05)  # Mix of ITM/OTM puts

    return K_prices


def generate_option_prices_for_idx(idx, n, weights=None, training_friendly_ratio=TFR_RATIO):

    rng = np.random.default_rng(idx)

    torch.manual_seed(idx)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(idx)

    S_0 = rng.uniform(MIN_PRICE_RANGE, MAX_PRICE_RANGE)

    # Generate option types
    option_types = rng.choice(["call", "put"], size=n)
    option_types_numeric = np.where(option_types == "call", 1, 0).astype(np.float32)

    # Generate hybrid strike prices
    K_prices = generate_hybrid_strikes(S_0, option_types, rng, TFR_RATIO)

    # Generate or use weights
    if weights is None:
        weight_sets = generate_combinatorial_weights_manageable(n)
        weights_array = weight_sets[0]
    else:
        weights_array = np.array(weights, dtype=np.float32)

    return K_prices, option_types_numeric, S_0, weights_array



def generate_combinatorial_weights_manageable(n, base_weights=[-0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75]):
    weight_sets = []

    # Handle the case where n < 2
    if n < 2:
        weights = np.zeros(n, dtype=np.float32)
        if n == 1:
            # If only one position, assign a long position (1.0)
            weights[0] = 1.0
        weight_sets.append(weights)
        return weight_sets

    # Generate a single portfolio: either one long or one short, and the rest from combinatorics
    weights = np.zeros(n, dtype=np.float32)

    # Randomly choose if we want a long or short portfolio
    is_long = random.choice([True, False])

    if is_long:
        # Choose one position to be long (1.0)
        long_idx = random.randint(0, n - 1)
        weights[long_idx] = 1.0
    else:
        # Choose one position to be short (-1.0)
        short_idx = random.randint(0, n - 1)
        weights[short_idx] = -1.0

    # Fill remaining positions with combinatorial weights from base_weights
    remaining_positions = [i for i in range(n) if weights[i] == 0]  # Find positions not yet filled
    combinatorics = np.random.choice(base_weights, size=len(remaining_positions), replace=True)

    # Assign combinatorial weights to the remaining positions without normalization
    weights[remaining_positions] = combinatorics

    weight_sets.append(weights)

    return weight_sets

def compute_cashflow_delta(S_T, portfolio):
    """
    Computes the delta of each option in the portfolio w.r.t terminal stock price S_T.

    Args:
        S_T (Tensor): [B] or [B, 1] terminal stock prices
        portfolio (Tensor): [B, N, 3] with columns (K, type, weight)

    Returns:
        Tensor: [B, N] delta for each option
    """
    # FIXED: Ensure S_T has correct shape
    if S_T.dim() > 2:
        S_T = S_T.squeeze()  # Remove extra dimensions
    if S_T.dim() == 2 and S_T.shape[1] == 1:
        S_T = S_T.squeeze(-1)  # [B, 1] → [B]

    strikes = portfolio[..., 0]  # [B, N]
    types = portfolio[..., 1]    # [B, N]

    # FIXED: Expand S_T to match portfolio shape [B, N]
    if S_T.dim() == 1:
        S_T_exp = S_T.unsqueeze(-1).expand(-1, strikes.shape[1])  # [B] → [B, N]
    else:
        S_T_exp = S_T.expand(-1, strikes.shape[1])  # [B, 1] → [B, N]

    # Call delta: 1 if S_T > K
    call_delta = ((types == 1) & (S_T_exp > strikes)).float()
    # Put delta: -1 if S_T < K
    put_delta = -((types == 0) & (S_T_exp < strikes)).float()

    delta_each = call_delta + put_delta  # [B, N]
    return delta_each

def compute_cashflow(portfolio, S_T):
    """
    Compute cashflow and portfolio-level derivative for a batch of portfolios.

    Args:
        portfolio (Tensor): [B, N, 3] tensor with (K, type, weight)
        S_T (Tensor): [B] or [B, 1] terminal stock prices

    Returns:
        cashflow (Tensor): [B] total weighted payoff per portfolio
        derivative (Tensor): [B] portfolio delta (sum of weighted option deltas)
    """
    # FIXED: Ensure S_T has correct shape
    if S_T.dim() > 2:
        S_T = S_T.squeeze()  # Remove extra dimensions
    if S_T.dim() == 2 and S_T.shape[1] == 1:
        S_T = S_T.squeeze(-1)  # [B, 1] → [B]

    strikes = portfolio[..., 0]  # [B, N]
    types = portfolio[..., 1]    # [B, N]
    weights = portfolio[..., 2]  # [B, N]

    # FIXED: Expand S_T to match portfolio shape [B, N]
    if S_T.dim() == 1:
        S_T_exp = S_T.unsqueeze(-1).expand(-1, strikes.shape[1])  # [B] → [B, N]
    else:
        S_T_exp = S_T.expand(-1, strikes.shape[1])  # [B, 1] → [B, N]

    # Compute payoffs
    call_payoffs = torch.relu(S_T_exp - strikes)  # [B, N]
    put_payoffs = torch.relu(strikes - S_T_exp)   # [B, N]
    payoffs = torch.where(types == 1, call_payoffs, put_payoffs)  # [B, N]

    # Weighted cashflow
    weighted_payoffs = payoffs * weights  # [B, N]
    cashflow = weighted_payoffs.sum(dim=1)  # [B]

    # Compute derivative (delta per option)
    delta_each = compute_cashflow_delta(S_T, portfolio)  # [B, N]
    weighted_delta = delta_each * weights  # [B, N]

    # Portfolio-level derivative
    derivative = weighted_delta.sum(dim=1)  # [B]

    return cashflow.float(), derivative.float()

def compute_cashflow_vectorized(portfolio, S_T_batch):
    """
    Vectorized computation for multiple S_T scenarios.

    Args:
        portfolio (Tensor): [B, N, 3] tensor with (K, type, weight)
        S_T_batch (Tensor): [B, M] multiple terminal prices per portfolio

    Returns:
        cashflow (Tensor): [B, M] cashflows for each scenario
        derivative (Tensor): [B, M] derivatives for each scenario
    """
    B, N, _ = portfolio.shape
    B2, M = S_T_batch.shape
    assert B == B2, f"Batch size mismatch: portfolio {B}, S_T {B2}"

    # Expand tensors for vectorized computation
    portfolio_exp = portfolio.unsqueeze(2).expand(-1, -1, M, -1)  # [B, N, M, 3]
    S_T_exp = S_T_batch.unsqueeze(1).expand(-1, N, -1)  # [B, N, M]

    strikes = portfolio_exp[..., 0]  # [B, N, M]
    types = portfolio_exp[..., 1]    # [B, N, M]
    weights = portfolio_exp[..., 2]  # [B, N, M]

    # Compute payoffs for all scenarios
    call_payoffs = torch.relu(S_T_exp - strikes)  # [B, N, M]
    put_payoffs = torch.relu(strikes - S_T_exp)   # [B, N, M]
    payoffs = torch.where(types == 1, call_payoffs, put_payoffs)  # [B, N, M]

    # Compute deltas for all scenarios
    call_delta = ((types == 1) & (S_T_exp > strikes)).float()  # [B, N, M]
    put_delta = -((types == 0) & (S_T_exp < strikes)).float()  # [B, N, M]
    delta_each = call_delta + put_delta  # [B, N, M]

    # Aggregate across options (sum over N dimension)
    cashflow = (payoffs * weights).sum(dim=1)  # [B, M]
    derivative = (delta_each * weights).sum(dim=1)  # [B, M]

    return cashflow.float(), derivative.float()

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset

class OperatorDatasetStandardized(Dataset):
    def __init__(self, num_samples, num_samples_S_T,
                 K_scaler=None, S_T_scaler=None, cashflow_scaler=None,
                 is_fitting_mode=False, max_portfolio_size=100, min_portfolio_size=1):
        self.num_samples = num_samples
        self.num_samples_S_T = num_samples_S_T
        self.max_portfolio_size = max_portfolio_size
        self.min_portfolio_size = min_portfolio_size
        self.is_fitting_mode = is_fitting_mode

        if not is_fitting_mode and any(s is None for s in (K_scaler, S_T_scaler, cashflow_scaler)):
            raise ValueError("K_scaler, S_T_scaler, and cashflow_scaler must be provided in evaluation mode.")

        self.K_scaler = K_scaler
        self.S_T_scaler = S_T_scaler
        self.cashflow_scaler = cashflow_scaler

    def __len__(self):
        return self.num_samples

    def _simulate_terminal_prices(self, S_0):
        # Example: Geometric Brownian Motion with noise
        Z = torch.clamp(torch.randn(self.num_samples_S_T), -3, 3)
        drift = (MU - 0.5 * SIGMA**2) * T
        diffusion = SIGMA * torch.sqrt(torch.tensor(T, dtype=torch.float32))
        S_T = S_0 * torch.exp(drift + diffusion * Z)
        S_T += torch.randn_like(S_T) * (NOISE_STD * S_T)
        return S_T.float()

    def __getitem__(self, idx):
        portfolio_len = np.random.randint(self.min_portfolio_size, self.max_portfolio_size + 1)
        K, option_types, S_0, weights = generate_option_prices_for_idx(idx, portfolio_len)

        # Ensure float32
        K = torch.tensor(K, dtype=torch.float32)
        option_types = torch.tensor(option_types, dtype=torch.float32)
        weights = torch.tensor(weights, dtype=torch.float32)
        S_0 = torch.tensor(S_0, dtype=torch.float32)

        # Portfolio tensor
        portfolio = torch.stack([K, option_types, weights], dim=-1)
        pad_len = self.max_portfolio_size - portfolio_len
        if pad_len > 0:
            pad_tensor = torch.zeros(pad_len, 3, dtype=torch.float32)
            portfolio = torch.cat([portfolio, pad_tensor], dim=0)

        mask = torch.tensor([True]*portfolio_len + [False]*pad_len, dtype=torch.bool)

        # Terminal prices - shape [num_samples_S_T]
        S_T = self._simulate_terminal_prices(S_0)

        # Store denormalized versions
        portfolio_denorm = portfolio.clone()
        S_T_denorm = S_T.clone()

        # FIXED: Use vectorized computation
        # Expand portfolio for batch computation: [1, max_portfolio_size, 3]
        portfolio_batch = portfolio_denorm.unsqueeze(0)  # Add batch dimension
        # Expand S_T for batch computation: [1, num_samples_S_T]
        S_T_batch = S_T_denorm.unsqueeze(0)  # Add batch dimension

        # Use vectorized computation
        cashflow_denorm, derivative_denorm = compute_cashflow_vectorized(
            portfolio_batch, S_T_batch
        )

        # Remove batch dimension: [1, M] → [M]
        cashflow_denorm = cashflow_denorm.squeeze(0).float()
        derivative_denorm = derivative_denorm.squeeze(0).float()

        if not self.is_fitting_mode:
            # Normalize only valid options
            valid_len = mask.sum().item()
            if valid_len > 0:
                K_norm = torch.tensor(
                    self.K_scaler.transform(K[:valid_len].unsqueeze(1)),
                    dtype=torch.float32
                ).squeeze()
                portfolio[:valid_len, 0] = K_norm

            # Normalize S_T: [M] → [M, 1] → transform → [M]
            S_T = torch.tensor(
                self.S_T_scaler.transform(S_T_denorm.unsqueeze(1)),
                dtype=torch.float32
            ).squeeze()

            # Normalize cashflow: [M] → [M, 1] → transform → [M]
            cashflow = torch.tensor(
                self.cashflow_scaler.transform(cashflow_denorm.unsqueeze(1)),
                dtype=torch.float32
            ).squeeze()

            # Scale derivative to match cashflow std
            derivative = derivative_denorm.clone()
            cf_std = cashflow.std().item()
            deriv_std = derivative.std().item()
            if deriv_std > 0:
                derivative = derivative * (cf_std / deriv_std)
        else:
            cashflow = cashflow_denorm.clone()
            derivative = derivative_denorm.clone()
            S_T = S_T_denorm.clone()

        return {
            "portfolio": portfolio.float(),          # [max_portfolio_size, 3]
            "mask": mask,                            # [max_portfolio_size]
            "S_T": S_T.float(),                      # [num_samples_S_T]
            "cashflow": cashflow.float(),            # [num_samples_S_T]
            "derivative": derivative.float(),        # [num_samples_S_T]
            "portfolio_denorm": portfolio_denorm.float(),
            "S_T_denorm": S_T_denorm.float(),
            "cashflow_denorm": cashflow_denorm.float(),
            "derivative_denorm": derivative_denorm.float()
        }



#Test with sample data

In [4]:
# --- Quick test run ---
dataset = OperatorDatasetStandardized(
    num_samples=2,
    num_samples_S_T=2,
    K_scaler=None, S_T_scaler=None, cashflow_scaler=None,
    max_portfolio_size=3,
    min_portfolio_size=1,
    is_fitting_mode=True
)

for i in range(len(dataset)):
    sample = dataset[i]  # dictionary returned
    print(f"\n=== Sample {i} ===")
    print("Portfolio features:\n", sample["portfolio"])
    print("Mask:", sample["mask"])
    print("S_T:", sample["S_T"])
    print("Cashflow:", sample["cashflow"])
    print("Derivative:", sample["derivative"])
    print("Portfolio (denorm):\n", sample["portfolio_denorm"])
    print("S_T (denorm):", sample["S_T_denorm"])
    print("Cashflow (denorm):", sample["cashflow_denorm"])
    print("Derivative (denorm):", sample["derivative_denorm"])



=== Sample 0 ===
Portfolio features:
 tensor([[ 1.5083e+03,  0.0000e+00,  2.5000e-01],
        [ 1.4125e+03,  1.0000e+00, -1.0000e+00],
        [ 1.3959e+03,  1.0000e+00, -7.5000e-01]])
Mask: tensor([True, True, True])
S_T: tensor([1812.2474, 1289.1338])
Cashflow: tensor([-711.9773,   54.7958])
Derivative: tensor([-1.7500, -0.2500])
Portfolio (denorm):
 tensor([[ 1.5083e+03,  0.0000e+00,  2.5000e-01],
        [ 1.4125e+03,  1.0000e+00, -1.0000e+00],
        [ 1.3959e+03,  1.0000e+00, -7.5000e-01]])
S_T (denorm): tensor([1812.2474, 1289.1338])
Cashflow (denorm): tensor([-711.9773,   54.7958])
Derivative (denorm): tensor([-1.7500, -0.2500])

=== Sample 1 ===
Portfolio features:
 tensor([[ 1.1459e+03,  0.0000e+00,  5.0000e-01],
        [ 1.4422e+03,  0.0000e+00, -1.0000e+00],
        [ 1.3374e+03,  1.0000e+00, -5.0000e-01]])
Mask: tensor([True, True, True])
S_T: tensor([1472.9598, 1368.7098])
Cashflow: tensor([-67.7839, -89.1250])
Derivative: tensor([-0.5000,  0.5000])
Portfolio (denorm)

In [5]:
import torch
import pytest

# Dummy constants for testing
MU = 0.05
T = 1.0
NOISE_STD = 0.0  # deterministic

# -------------------------
# Minimal compute_cashflow
# -------------------------
def compute_cashflow(portfolio_batch, S_T_batch):
    """
    portfolio_batch: [B, N, 3] tensor
    S_T_batch: [B, 1] tensor
    Returns: cashflow [B], derivative [B]
    """
    B, N, _ = portfolio_batch.shape
    cashflow = torch.zeros(B)
    derivative = torch.zeros(B)

    for i in range(B):
        total = 0.0
        delta_total = 0.0
        for j in range(N):
            K, t, w = portfolio_batch[i, j]
            s = S_T_batch[i, 0]
            if t == 1:  # call
                payoff = max(s - K, 0)
                delta = 1.0 if s > K else 0.0
            else:       # put
                payoff = max(K - s, 0)
                delta = -1.0 if s > K else 0.0
            total += w * payoff
            delta_total += w * delta
        cashflow[i] = total
        derivative[i] = delta_total
    return cashflow, derivative

# -------------------------
# Manual reference for test
# -------------------------
def manual_cashflow_delta(S_T, portfolio):
    cashflow = []
    derivative = []
    for s in S_T:
        total = 0.0
        delta_total = 0.0
        for k, t, w in portfolio:
            if t == 1:  # call
                payoff = max(s - k, 0)
                delta = 1.0 if s > k else 0.0
            else:       # put
                payoff = max(k - s, 0)
                delta = -1.0 if s > k else 0.0
            total += w * payoff
            delta_total += w * delta
        cashflow.append(total)
        derivative.append(delta_total)
    return torch.tensor(cashflow, dtype=torch.float32), torch.tensor(derivative, dtype=torch.float32)

# -------------------------
# Test cashflow/derivative for known portfolio
# -------------------------
def test_cashflow_derivative_manual():
    portfolio = torch.tensor([[100.0, 1.0, 1.0], [120.0, 0.0, -0.5]], dtype=torch.float32)
    S_T = torch.tensor([90.0, 110.0, 130.0], dtype=torch.float32)

    portfolio_batch = portfolio.unsqueeze(0).expand(len(S_T), -1, -1)
    cashflow, derivative = compute_cashflow(portfolio_batch, S_T.unsqueeze(-1))
    expected_cashflow, expected_derivative = manual_cashflow_delta(S_T, portfolio)

    assert torch.allclose(cashflow.squeeze(), expected_cashflow, atol=1e-4)
    assert torch.allclose(derivative.squeeze(), expected_derivative, atol=1e-4)
    print("✅ Cashflow and derivative tests passed!")

# -------------------------
# Test dataset integration
# -------------------------
class DummyDataset:
    def __getitem__(self, idx):
        portfolio = torch.tensor([[100.0, 1.0, 1.0], [120.0, 0.0, -0.5]], dtype=torch.float32)
        S_T = torch.tensor([90.0, 110.0, 130.0], dtype=torch.float32)
        portfolio_batch = portfolio.unsqueeze(0).expand(len(S_T), -1, -1)
        cashflow, derivative = compute_cashflow(portfolio_batch, S_T.unsqueeze(-1))
        mask = torch.tensor([True, True], dtype=torch.bool)
        # Return same signature as OperatorDatasetStandardized
        return {
            "portfolio": portfolio,
            "mask": mask,
            "S_T": S_T,
            "cashflow": cashflow.squeeze(),
            "derivative": derivative.squeeze(),
            "portfolio_denorm": portfolio,
            "S_T_denorm": S_T,
            "cashflow_denorm": cashflow.squeeze(),
            "derivative_denorm": derivative.squeeze()
        }

def test_dataset_cashflow_derivative():
    dataset = DummyDataset()
    sample = dataset[0]
    expected_cashflow, expected_derivative = manual_cashflow_delta(sample["S_T"], sample["portfolio"])
    assert torch.allclose(sample["cashflow"], expected_cashflow, atol=1e-4)
    assert torch.allclose(sample["derivative"], expected_derivative, atol=1e-4)
    print("✅ Dataset cashflow and derivative tests passed!")

# Run tests
test_cashflow_derivative_manual()
test_dataset_cashflow_derivative()


✅ Cashflow and derivative tests passed!
✅ Dataset cashflow and derivative tests passed!


In [6]:
import os
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
from tqdm import tqdm

DRIVE_PATH = "/content/drive/MyDrive/Ucl/"
K_SCALAR_FILE = os.path.join(DRIVE_PATH, 'K_Scalar_Advanced.pkl')
ST_SCALAR_FILE = os.path.join(DRIVE_PATH, 'S_T_Scalar_Advanced.pkl')
CASHFLOW_SCALAR_FILE = os.path.join(DRIVE_PATH, 'Cashflow_Scalar_Advanced.pkl')


def fit_K_ST_scalers(train_loader, save_path_K=K_SCALAR_FILE, save_path_ST=ST_SCALAR_FILE):
    print("Fitting K and S_T scalers from training set...")
    all_K = []
    all_S_T = []

    for batch in tqdm(train_loader, desc="Collecting K and S_T for scalers"):
        # Access dict keys safely
        portfolio_real = batch["portfolio"]  # [B, N, 3]
        s_t_real = batch["S_T"]              # [B, num_S_T]

        # K is the first column of the portfolio features
        K_real = portfolio_real[:, :, 0].cpu().numpy().reshape(-1, 1)
        S_T_real = s_t_real.cpu().numpy().reshape(-1, 1)

        all_K.append(K_real)
        all_S_T.append(S_T_real)

    K_all_np = np.concatenate(all_K, axis=0)
    S_T_all_np = np.concatenate(all_S_T, axis=0)

    K_scalar = StandardScaler()
    K_scalar.fit(K_all_np)
    #joblib.dump(K_scalar, save_path_K)

    S_T_scalar = StandardScaler()
    S_T_scalar.fit(S_T_all_np)
    #joblib.dump(S_T_scalar, save_path_ST)

    # print(f"✅ Saved K scalar to: {save_path_K}")
    # print(f"✅ Saved S_T scalar to: {save_path_ST}")
    # print(f"K mean: {K_scalar.mean_[0]:.4f}, std: {K_scalar.scale_[0]:.4f}")
    # print(f"S_T mean: {S_T_scalar.mean_[0]:.4f}, std: {S_T_scalar.scale_[0]:.4f}")

    return K_scalar, S_T_scalar


def fit_cashflow_scaler(train_loader, save_path=CASHFLOW_SCALAR_FILE):
    all_cashflows = []

    for batch in tqdm(train_loader, desc="Fitting Cashflow Scaler"):
        cashflow = batch["cashflow"]  # [B, N] or [B*num_S_T]
        cashflow_np = cashflow.detach().cpu().numpy().reshape(-1, 1)
        all_cashflows.append(cashflow_np)

    cashflows_np = np.concatenate(all_cashflows, axis=0)

    scaler = StandardScaler()
    scaler.fit(cashflows_np)
    #joblib.dump(scaler, save_path)

    # print(f"✅ Saved Cashflow Scaler to: {save_path}")
    # print(f"Cashflow Mean: {scaler.mean_[0]:.4f}, Std Dev: {scaler.scale_[0]:.4f}")

    return scaler


In [7]:
import torch
import torch.nn as nn
import numpy as np
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from tqdm import tqdm

FEED_FWD_DEPTH = 3


DRPO_OUT_PROB = 0.1


class TrunkNet(nn.Module):
    def __init__(self, input_dim=1, latent_dim=64, hidden_dim=128,
                 num_layers=6, dropout_prob=DRPO_OUT_PROB):
        super(TrunkNet, self).__init__()

        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        self.blocks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout_prob)
            ) for _ in range(num_layers)
        ])

        self.output_proj = nn.Linear(hidden_dim, latent_dim)

    def forward(self, S_T):
        if S_T.dim() == 1:
            S_T = S_T.unsqueeze(-1)
        elif S_T.dim() == 2:
            S_T = S_T.unsqueeze(-1)

        x = self.input_proj(S_T)
        for block in self.blocks:
            x = x + block(x)
        return self.output_proj(x)


class ISAB(nn.Module):
    """Induced Set Attention Block"""
    def __init__(self, d_model, num_heads, num_inds, dropout=DRPO_OUT_PROB):
        super().__init__()
        self.num_inds = num_inds
        self.inducing_points = nn.Parameter(torch.randn(num_inds, d_model))

        self.attention1 = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.attention2 = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)

        self.ffn1 = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )
        self.ffn2 = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask=None):
        batch_size = x.size(0)
        I = self.inducing_points.unsqueeze(0).expand(batch_size, -1, -1)

        # I attends to X
        attn_out1, _ = self.attention1(
            I, x, x,
            key_padding_mask=(~mask.bool()) if mask is not None else None
        )
        I = self.norm1(I + attn_out1)
        I = self.norm2(I + self.ffn1(I))

        # X attends to I
        attn_out2, _ = self.attention2(x, I, I)
        x = self.norm3(x + attn_out2)
        x = self.norm4(x + self.ffn2(x))

        return x


class PMA(nn.Module):
    """Pooling by Multihead Attention"""
    def __init__(self, d_model, num_heads, num_seeds, dropout=DRPO_OUT_PROB):
        super().__init__()
        self.num_seeds = num_seeds
        self.seed_vectors = nn.Parameter(torch.randn(num_seeds, d_model))

        self.attention = nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * FEED_FWD_DEPTH),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * FEED_FWD_DEPTH, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask=None):
        batch_size = x.size(0)
        S = self.seed_vectors.unsqueeze(0).expand(batch_size, -1, -1)

        key_padding_mask = (~mask.bool()) if mask is not None else None

        attn_out, _ = self.attention(S, x, x, key_padding_mask=key_padding_mask)
        S = self.norm1(S + attn_out)
        S = self.norm2(S + self.ffn(S))
        return S


class SAB(nn.Module):
    """Set Attention Block"""
    def __init__(self, d_model, num_heads, dropout=DRPO_OUT_PROB):
        super().__init__()
        self.transformer_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=d_model * FEED_FWD_DEPTH,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )

    def forward(self, x, mask=None):
        if mask is not None:
            src_key_padding_mask = ~mask.bool()
        else:
            src_key_padding_mask = None
        return self.transformer_layer(x, src_key_padding_mask=src_key_padding_mask)


class EnhancedSetTransformerEncoder(nn.Module):
    """Set Transformer with ISAB + PMA"""
    def __init__(self, portfolio_feature_dim=3, latent_dim=128, hidden_dim=64,
                 num_heads=2, dropout_prob=DRPO_OUT_PROB, num_inds=32, num_seeds=1,
                 use_isab=True, num_layers=2):
        super().__init__()

        if hidden_dim % num_heads != 0:
            hidden_dim = ((hidden_dim // num_heads) + 1) * num_heads

        self.input_proj = nn.Sequential(
            nn.Linear(portfolio_feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        self.encoder_layers = nn.ModuleList()
        for _ in range(num_layers):
            if use_isab:
                self.encoder_layers.append(ISAB(hidden_dim, num_heads, num_inds, dropout_prob))
            else:
                self.encoder_layers.append(SAB(hidden_dim, num_heads, dropout_prob))

        self.pooling = PMA(hidden_dim, num_heads, num_seeds, dropout_prob)

        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, portfolio, mask=None):
        x = self.input_proj(portfolio)  # [B, P, H]
        for layer in self.encoder_layers:
            x = layer(x, mask=mask)
        x = self.pooling(x, mask=mask)
        x = x.mean(dim=1) if x.size(1) > 1 else x.squeeze(1)
        return self.output_proj(x)


class OptimizedSetTransformerEncoder(nn.Module):
    """Vanilla Set Transformer with mask support"""
    def __init__(self, portfolio_feature_dim=3, latent_dim=128, hidden_dim=64,
                 num_layers=1, num_heads=2, dropout_prob=DRPO_OUT_PROB):
        super().__init__()

        if hidden_dim % num_heads != 0:
            hidden_dim = ((hidden_dim // num_heads) + 1) * num_heads

        self.input_proj = nn.Sequential(
            nn.Linear(portfolio_feature_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * FEED_FWD_DEPTH,
            dropout=dropout_prob,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
            enable_nested_tensor=False
        )

        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(dropout_prob),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, portfolio, mask=None):
        x = self.input_proj(portfolio)
        src_key_padding_mask = ~mask.bool() if mask is not None else None
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        pooled = x.mean(dim=1) + x.max(dim=1).values
        return self.output_proj(pooled)


class OptimizedDeepONet(nn.Module):
    """DeepONet with branch (portfolio) + trunk (S_T)"""
    def __init__(self, portfolio_feature_dim=3, hidden_dim=64, latent_dim=128,
                 dropout_prob=DRPO_OUT_PROB, num_heads=2, use_enhanced_transformer=True,
                 num_inds=32, num_seeds=1):
        super().__init__()

        if hidden_dim % num_heads != 0:
            recommended = ((hidden_dim // num_heads) + 1) * num_heads
            raise ValueError(
                f"hidden_dim ({hidden_dim}) must be divisible by num_heads ({num_heads}). "
                f"Try hidden_dim={recommended}"
            )

        if use_enhanced_transformer:
            self.branch_net = EnhancedSetTransformerEncoder(
                portfolio_feature_dim=portfolio_feature_dim,
                latent_dim=latent_dim,
                hidden_dim=hidden_dim,
                num_heads=num_heads,
                dropout_prob=dropout_prob,
                num_inds=num_inds,
                num_seeds=num_seeds,
                use_isab=True,
                num_layers=2
            )
        else:
            self.branch_net = OptimizedSetTransformerEncoder(
                portfolio_feature_dim=portfolio_feature_dim,
                latent_dim=latent_dim,
                hidden_dim=hidden_dim,
                dropout_prob=dropout_prob,
                num_heads=num_heads
            )

        self.trunk_net = TrunkNet(
            input_dim=1,
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            dropout_prob=dropout_prob
        )

        self.bias = nn.Parameter(torch.zeros(1))
        self.branch_scale = nn.Parameter(torch.ones(1) * 0.8)
        self.trunk_scale = nn.Parameter(torch.ones(1) * 0.8)

    def forward(self, portfolio, S_T, mask=None):
        B, M = S_T.shape

        branch_out = self.branch_net(portfolio, mask=mask) * self.branch_scale

        trunk_outputs = []
        for i in range(M):
            S_T_single = S_T[:, i:i+1]
            trunk_out = self.trunk_net(S_T_single) * self.trunk_scale
            trunk_outputs.append(trunk_out)

        trunk_out = torch.cat(trunk_outputs, dim=1)  # [B, M, latent_dim]

        branch_expanded = branch_out.unsqueeze(1)  # [B, 1, latent_dim]
        interaction = (branch_expanded * trunk_out).sum(dim=-1)  # [B, M]

        return interaction + self.bias


        # branch_out = self.branch_net(portfolio, mask=mask) * self.branch_scale
        # trunk_out = self.trunk_net(S_T) * self.trunk_scale
        # branch_expanded = branch_out.unsqueeze(1)
        # interaction = (branch_expanded * trunk_out).sum(dim=-1)
        # return interaction + self.bias


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import GradScaler

# --- Gradient summary helper ---
def compute_gradient_stats(model):
    gradient_stats = {}
    total_norm = 0.0
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm().item()
            total_norm += grad_norm ** 2
            gradient_stats[name] = {
                'norm': grad_norm,
                'shape': tuple(param.grad.shape),
                'numel': param.grad.numel(),
                'mean': param.grad.mean().item(),
                'std': param.grad.std().item()
            }
    total_norm = total_norm ** 0.5
    return total_norm, gradient_stats, None

def print_gradient_summary(gradient_stats, total_norm, epoch, batch_idx=None):
    prefix = f"Epoch {epoch}" + (f", Batch {batch_idx}" if batch_idx is not None else "")
    print(f"\n🔍 === Gradient Analysis - {prefix} ===")
    print(f"Total Gradient Norm: {total_norm:.6f}")

    if total_norm > 30.0:
        print("🚨 CRITICAL: Severe gradient explosion! Consider stopping training.")
    elif total_norm > 20.0:
        print("⚠️  SEVERE: Major gradient explosion detected!")
    elif total_norm > 10.0:
        print("⚠️  WARNING: Moderate gradient explosion detected!")
    elif total_norm < 1e-6:
        print("⚠️  WARNING: Vanishing gradients detected!")
    else:
        print("✅ Gradient norm is healthy")

    sorted_layers = sorted(gradient_stats.items(), key=lambda x: x[1]['norm'], reverse=True)
    print(f"\nTop 5 layers by gradient norm (out of {len(gradient_stats)} total):")
    for i, (layer_name, stats) in enumerate(sorted_layers[:5]):
        status = "🔥" if stats['norm'] > 3.0 else "⚠️" if stats['norm'] > 1.0 else "✅"
        print(f"  {status} {i+1}. {layer_name}: {stats['norm']:.4f}")
        print(f"      Shape: {stats['shape']}, Elements: {stats['numel']}")
        print(f"      Mean: {stats['mean']:.6f}, Std: {stats['std']:.6f}")
    print("=" * 60)


In [9]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

class ExtendedEarlyStopping:
    # ... (no changes needed here) ...
    def __init__(self, patience=30, min_delta=0.0005, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.wait = 0
        self.stopped_epoch = 0
        self.best = float('inf')
        self.best_weights = None

    def __call__(self, val_loss, model=None):
        if val_loss < self.best - self.min_delta:
            self.best = val_loss
            self.wait = 0
            if model is not None and self.restore_best_weights:
                self.best_weights = model.state_dict().copy()
        else:
            self.wait += 1

        if self.wait >= self.patience:
            self.stopped_epoch = True
            if model is not None and self.restore_best_weights and self.best_weights is not None:
                model.load_state_dict(self.best_weights)

        return self.stopped_epoch


In [99]:
def get_stable_hyperparameters():
    """Return more stable hyperparameters"""
    return {
        "learning_rate": 3e-4,
        "weight_decay": 1e-4 ,
        "lambda_deriv": 0.05,
        "lambda_reg": 1e-4,
        "gradient_clip_norm": 5,
        "batch_size": 128,
        "scheduler_T0": 5,
        "early_stopping_patience": 35,
    }


In [100]:

class OptimizedTrainer:
    def __init__(self, model, device='cuda', monitor_gradients=True,
                 learning_rate=5e-6, lambda_deriv_weight=0.1, weight_decay=1e-4,
                 scale_warmup_epochs=5, initial_scale=0.05, final_scale=1.0, grad_log_threshold = 5.0):
        self.model = model.to(device)
        self.device = device
        self.monitor_gradients = monitor_gradients
        self.lambda_deriv_weight = lambda_deriv_weight

        # Optimizer
        self.optimizer = optim.AdamW(
            model.parameters(),
            lr=learning_rate,
            weight_decay=weight_decay,
            betas=(0.9, 0.999),
            eps=1e-8
        )

        # Cosine Annealing Warm Restarts scheduler (epoch-based)
        self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=scale_warmup_epochs, T_mult=1, eta_min=1e-6
        )

        # Gradient scaler for mixed precision
        self.scaler = GradScaler()
        self.huber_loss = nn.SmoothL1Loss(beta=1.0)

        # Branch/trunk scale
        self.scale_warmup_epochs = scale_warmup_epochs
        self.initial_scale = initial_scale
        self.final_scale = final_scale
        if hasattr(self.model, 'branch_scale') and hasattr(self.model, 'trunk_scale'):
            with torch.no_grad():
                self.model.branch_scale.fill_(initial_scale)
                self.model.trunk_scale.fill_(initial_scale)

    def check_model_health(self, epoch, batch_idx):
        for name, param in self.model.named_parameters():
            if torch.isnan(param).any() or torch.isinf(param).any():
                print(f"Bad parameter: {name} at Epoch {epoch}, Batch {batch_idx}")
                return False
        return True

    def compute_loss(self, pred_cashflow, true_cashflow, pred_deriv=None, true_deriv=None, mask=None):
        """
        FIXED: Computes total loss correctly without misapplying masks.

        Key insight: Portfolio mask is for MODEL INPUTS (which options are valid),
        not for LOSS COMPUTATION (all scenarios are always valid).

        Args:
            pred_cashflow: [B, M] predicted cashflows for M scenarios
            true_cashflow: [B, M] true cashflows for M scenarios
            pred_deriv: [B, M] predicted derivatives for M scenarios
            true_deriv: [B, M] true derivatives for M scenarios
            mask: [B, N] portfolio mask (NOT used in loss computation)
        """

        # --- Cashflow loss (no masking needed) ---
        # All scenarios are always valid, regardless of portfolio structure
        cashflow_loss = self.huber_loss(pred_cashflow, true_cashflow)

        # --- Derivative loss (no masking needed) ---
        if pred_deriv is not None and true_deriv is not None:
            deriv_loss = self.huber_loss(pred_deriv, true_deriv)
            total_loss = cashflow_loss + self.lambda_deriv_weight * deriv_loss
        else:
            deriv_loss = torch.tensor(0.0, device=pred_cashflow.device)
            total_loss = cashflow_loss

        return total_loss, cashflow_loss, deriv_loss

    def train_step(self, portfolio, S_T, cashflow, true_derivative=None, mask=None, epoch=0, batch_idx=0, log_gradients=False):
      self.optimizer.zero_grad()

      S_T = S_T.clone().detach().requires_grad_(True).to(self.device)
      portfolio = portfolio.to(self.device)
      cashflow = cashflow.to(self.device)
      if mask is not None:
          mask = mask.to(self.device)
      if true_derivative is not None:
          true_derivative = true_derivative.to(self.device)

      pred_cashflow = self.model(portfolio, S_T, mask=mask)

      pred_deriv = None
      if true_derivative is not None:
          # FIXED: Compute gradients per scenario, not summed
          pred_deriv = torch.autograd.grad(
              outputs=pred_cashflow,
              inputs=S_T,
              grad_outputs=torch.ones_like(pred_cashflow),  # ADD THIS LINE
              retain_graph=True,
              create_graph=True,
              allow_unused=True
          )[0]

      total_loss, cashflow_loss, deriv_loss = self.compute_loss(
          pred_cashflow, cashflow, pred_deriv, true_derivative, mask=None
      )

      self.scaler.scale(total_loss).backward()
      self.scaler.unscale_(self.optimizer)
      torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=5.0)
      self.scaler.step(self.optimizer)
      self.scaler.update()

      return total_loss.item(), cashflow_loss.item(), deriv_loss.item()

    def val_step(self, portfolio, S_T, cashflow, true_derivative=None, mask=None):
      """
      Validation step that can compute derivative loss as well.
      """
      self.model.eval()

      portfolio = portfolio.to(self.device)
      S_T = S_T.clone().detach().requires_grad_(True).to(self.device)  # need grad for derivative
      cashflow = cashflow.to(self.device)
      if mask is not None:
          mask = mask.to(self.device)
      if true_derivative is not None:
          true_derivative = true_derivative.to(self.device)

      # always compute forward with grad enabled (so we can use for both losses)
      with torch.enable_grad():
          pred_cashflow = self.model(portfolio, S_T, mask=mask)

          pred_deriv = None
          if true_derivative is not None:
              pred_deriv = torch.autograd.grad(
                  outputs=pred_cashflow,
                  inputs=S_T,
                  grad_outputs=torch.ones_like(pred_cashflow),
                  retain_graph=False,
                  create_graph=False,
                  allow_unused=True
              )[0]

      # detach before computing loss to avoid holding graph in memory
      total_loss, cashflow_loss, deriv_loss = self.compute_loss(
          pred_cashflow.detach(), cashflow,
          pred_deriv.detach() if pred_deriv is not None else None,
          true_derivative, mask=None
      )

      return total_loss.item(), cashflow_loss.item(), deriv_loss.item()


    def update_scale(self, current_epoch):
        """Update branch/trunk scale during warmup."""
        if hasattr(self.model, 'branch_scale') and hasattr(self.model, 'trunk_scale'):
            factor = min((current_epoch + 1) / self.scale_warmup_epochs, 1.0)
            new_scale = self.initial_scale + (self.final_scale - self.initial_scale) * factor
            with torch.no_grad():
                self.model.branch_scale.fill_(new_scale)
                self.model.trunk_scale.fill_(new_scale)
            print(f"[Epoch {current_epoch}] Updated branch/trunk scale → {new_scale:.4f}")

    def step_scheduler_epoch(self):
        """Step scheduler once per epoch."""
        self.scheduler.step()


In [60]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : interesting_taper_4184
COMET INFO:     url                   : https://www.comet.com/satyabratkumarsingh/option-portfolio-encoder-decoder/10838a5cf05d46fab30167e6bb067314
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_cashflow_loss_batch [1120]   : (0.10014146566390991, 0.71085125207901)
COMET INFO:     train_cashflow_loss_epoch [7]      : (0.1341879060026258, 0.2814498905092478)
COMET INFO:     train_derivative_loss_batch [1120] : (0.3015000820159912, 1.0325230360031128)
COMET INFO:     train_derivative_loss_epoch [7]    : (0.45259464904665947, 0.6696506692096591)
COMET INFO:     train_total_loss_batch [1120]      : (0.1192431300878

# Model save

In [101]:
def save_model_checkpoint(model, save_path, epoch=None, optimizer=None, scheduler=None,
                         train_loss=None, val_loss=None, train_size=None, val_size=None):

    checkpoint_data = {
        "model_state_dict": model.state_dict(),
        "hparams": {
            "hidden_dim": hidden_dim,
            "latent_dim": latent_dim,
            "portfolio_feature_dim": portfolio_feature_dim,
            "use_enhanced_transformer": True
        },
        "training_config": {
            "PORT_LEN": PORT_LEN,
            "PORT_SAMPLE_SIZE": PORT_SAMPLE_SIZE,
            "FEED_ST_LEN_EACH_PORT": FEED_ST_LEN_EACH_PORT,
            "batch_size": batch_size,
            "train_size": train_size,
            "val_size": val_size
        },
        "scaler_files": {
            "K_scaler": "K_Scalar_Training.pkl",
            "S_T_scaler": "S_T_Scalar_Training.pkl",
            "cashflow_scaler": "Cashflow_Scalar_Training.pkl"
        }
    }

    # Add optional training state information
    if epoch is not None:
        checkpoint_data["epoch"] = epoch
    if optimizer is not None:
        checkpoint_data["optimizer_state_dict"] = optimizer.state_dict()
    if scheduler is not None:
        checkpoint_data["scheduler_state_dict"] = scheduler.state_dict()
    if train_loss is not None:
        checkpoint_data["train_loss"] = train_loss
    if val_loss is not None:
        checkpoint_data["val_loss"] = val_loss

    torch.save(checkpoint_data, save_path)

In [102]:
import torch
from torch.utils.data import DataLoader, random_split
import numpy as np
import joblib  # Add joblib for saving scalers
from comet_ml import start
from tqdm import tqdm
import os  # Add for directory creation

# === HYPERPARAMETERS ===
hidden_dim = 128
latent_dim = 128
batch_size = 128
epochs = 500
portfolio_feature_dim = 3
PORT_LEN = 100
PORT_SAMPLE_SIZE = 25600
FEED_ST_LEN_EACH_PORT = 100

# === COMET SETUP ===
experiment = start(
    api_key="iatWnXT4JyBtDQhn7OfgISQoF",
    project_name="option-portfolio-encoder-decoder",
    workspace="satyabratkumarsingh"
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def main():
    """
    FIXED: Corrected main training loop with proper validation computation.
    Added scaler saving for consistent evaluation.
    Added model saving every 50 epochs with -epoch{no} suffix.
    """
    hparams = get_stable_hyperparameters()
    experiment.log_parameters(hparams)

    # === Model ===
    model = OptimizedDeepONet(
        portfolio_feature_dim=portfolio_feature_dim,
        hidden_dim=hidden_dim,
        latent_dim=latent_dim,
        use_enhanced_transformer=True
    ).to(DEVICE)

    # --- STEP 1: Raw dataset for scaler fitting ---
    raw_dataset = OperatorDatasetStandardized(
        num_samples=PORT_SAMPLE_SIZE,
        min_portfolio_size=1,
        max_portfolio_size=PORT_LEN,
        num_samples_S_T=FEED_ST_LEN_EACH_PORT,
        is_fitting_mode=True
    )

    # Train/Val split
    train_size = int(0.8 * len(raw_dataset))
    val_size = len(raw_dataset) - train_size
    val_size = (val_size // batch_size) * batch_size
    train_size = len(raw_dataset) - val_size

    torch.manual_seed(42)
    train_dataset, val_dataset = random_split(raw_dataset, [train_size, val_size])
    train_loader_fitting = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    # Fit scalers
    print("Fitting scalers on training data...")
    K_scalar, S_T_scalar = fit_K_ST_scalers(train_loader_fitting)
    cashflow_scaler = fit_cashflow_scaler(train_loader_fitting)

    # === SAVE SCALERS ===
    DRIVE_PATH = "/content/drive/MyDrive/Ucl/"
    print("Saving scalers for evaluation consistency...")

    # Create checkpoint directory for periodic model saves
    checkpoint_dir = DRIVE_PATH + "checkpoints/"
    os.makedirs(checkpoint_dir, exist_ok=True)

    try:
        # Save scalers with consistent naming
        joblib.dump(K_scalar, DRIVE_PATH + "K_Scalar_Training.pkl")
        joblib.dump(S_T_scalar, DRIVE_PATH + "S_T_Scalar_Training.pkl")
        joblib.dump(cashflow_scaler, DRIVE_PATH + "Cashflow_Scalar_Training.pkl")
        print("✅ Successfully saved all training scalers")

        # Print scaler statistics for debugging
        print(f"📊 Scaler Statistics:")
        print(f"K_scaler - mean: {K_scalar.mean_[0]:.4f}, std: {np.sqrt(K_scalar.var_[0]):.4f}")
        print(f"S_T_scaler - mean: {S_T_scalar.mean_[0]:.4f}, std: {np.sqrt(S_T_scalar.var_[0]):.4f}")
        print(f"Cashflow_scaler - mean: {cashflow_scaler.mean_[0]:.4f}, std: {np.sqrt(cashflow_scaler.var_[0]):.4f}")

    except Exception as e:
        print(f"❌ Error saving scalers: {e}")
        print("Continuing with training but evaluation may be inconsistent...")

    # --- STEP 2: Normalized dataset ---
    normalized_dataset = OperatorDatasetStandardized(
        num_samples=PORT_SAMPLE_SIZE,
        min_portfolio_size=1,  # Add missing parameter for consistency
        max_portfolio_size=PORT_LEN,
        num_samples_S_T=FEED_ST_LEN_EACH_PORT,
        K_scaler=K_scalar,
        S_T_scaler=S_T_scalar,
        cashflow_scaler=cashflow_scaler,
        is_fitting_mode=False
    )

    torch.manual_seed(42)
    train_dataset, val_dataset = random_split(normalized_dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # === Trainer ===
    trainer = OptimizedTrainer(
        model,
        device=DEVICE,
        learning_rate=hparams["learning_rate"],
        lambda_deriv_weight=hparams["lambda_deriv"],
        weight_decay=hparams["weight_decay"],
        monitor_gradients=True,
        grad_log_threshold=5.0,
        scale_warmup_epochs=hparams["scheduler_T0"],
        initial_scale=0.05,
        final_scale=1.0
    )

    early_stopper = ExtendedEarlyStopping(
        patience=hparams["early_stopping_patience"],
        min_delta=0.001,
        restore_best_weights=True
    )

    # === Training loop ===
    for epoch in range(epochs):
        trainer.update_scale(epoch)
        model.train()

        # --- Training Loop ---
        train_total_losses, train_cf_losses, train_deriv_losses = [], [], []

        for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch}")):
            portfolio = batch["portfolio"].to(DEVICE)
            mask = batch["mask"].to(DEVICE)
            S_T = batch["S_T"].to(DEVICE)
            cashflow = batch["cashflow"].to(DEVICE)
            derivative = batch["derivative"].to(DEVICE)

            # CORRECT: Training step computes and returns training losses
            total, cf, deriv = trainer.train_step(
                portfolio,
                S_T.clone().detach().requires_grad_(True),
                cashflow,
                derivative,
                mask
            )

            train_total_losses.append(total)
            train_cf_losses.append(cf)
            train_deriv_losses.append(deriv)

            # --- Batch-level logging ---
            global_step = epoch * len(train_loader) + batch_idx
            experiment.log_metric("train_total_loss_batch", total, step=global_step)
            experiment.log_metric("train_cashflow_loss_batch", cf, step=global_step)
            experiment.log_metric("train_derivative_loss_batch", deriv, step=global_step)

        # --- Epoch-level metrics ---
        avg_train_total = np.mean(train_total_losses)
        avg_train_cf = np.mean(train_cf_losses)
        avg_train_deriv = np.mean(train_deriv_losses)

        experiment.log_metric("train_total_loss_epoch", avg_train_total, step=epoch)
        experiment.log_metric("train_cashflow_loss_epoch", avg_train_cf, step=epoch)
        experiment.log_metric("train_derivative_loss_epoch", avg_train_deriv, step=epoch)

        # --- FIXED: Validation Loop ---
        model.eval()
        val_total_losses, val_cf_losses, val_deriv_losses = [], [], []

        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                portfolio = batch["portfolio"].to(DEVICE)
                mask = batch["mask"].to(DEVICE)
                S_T = batch["S_T"].to(DEVICE)
                cashflow = batch["cashflow"].to(DEVICE)
                derivative = batch["derivative"].to(DEVICE)

                # FIXED: Actually compute validation losses using val_step
                total, cf, deriv = trainer.val_step(
                    portfolio, S_T, cashflow, derivative, mask
                )

                val_total_losses.append(total)
                val_cf_losses.append(cf)
                val_deriv_losses.append(deriv)

                # --- Batch-level validation logging ---
                global_step = epoch * len(val_loader) + batch_idx
                experiment.log_metric("val_total_loss_batch", total, step=global_step)
                experiment.log_metric("val_cashflow_loss_batch", cf, step=global_step)
                experiment.log_metric("val_derivative_loss_batch", deriv, step=global_step)

        # Step the scheduler once per epoch
        trainer.step_scheduler_epoch()

        current_lr = trainer.optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch} → Current LR: {current_lr:.6e}")

        # --- Epoch-level validation metrics ---
        avg_val_total = np.mean(val_total_losses)
        avg_val_cf = np.mean(val_cf_losses)
        avg_val_deriv = np.mean(val_deriv_losses)

        experiment.log_metric("val_total_loss_epoch", avg_val_total, step=epoch)
        experiment.log_metric("val_cashflow_loss_epoch", avg_val_cf, step=epoch)
        experiment.log_metric("val_derivative_loss_epoch", avg_val_deriv, step=epoch)

        print(
            f"Epoch [{epoch}/{epochs}] "
            f"Train → total: {avg_train_total:.6f}, cf: {avg_train_cf:.6f}, deriv: {avg_train_deriv:.6f} | "
            f"Val → total: {avg_val_total:.6f}, cf: {avg_val_cf:.6f}, deriv: {avg_val_deriv:.6f}"
        )

        # === SAVE MODEL EVERY 50 EPOCHS ===
        if (epoch + 1) % 50 == 0:  # epoch starts from 0, so epoch+1 for proper numbering
            checkpoint_path = checkpoint_dir + f"deeponet_model-epoch{epoch + 1}.pt"

            # Save model checkpoint using the reusable function
            save_model_checkpoint(
                model=model,
                save_path=checkpoint_path,
                epoch=epoch + 1,
                optimizer=trainer.optimizer,
                scheduler=trainer.scheduler if hasattr(trainer, 'scheduler') else None,
                train_loss=avg_train_total,
                val_loss=avg_val_total
            )

            print(f"🔄 Model checkpoint saved at epoch {epoch + 1}: {checkpoint_path}")

        # --- Early Stopping Check ---
        stop = early_stopper(avg_val_total, model)
        if stop:
            print(f"Early stopping triggered at epoch {epoch}. Best val loss: {early_stopper.best:.6f}")
            break

    # === SAVE FINAL MODEL WITH METADATA ===
    save_path = DRIVE_PATH + "final_deeponet_model.pt"

    # Save final model using the reusable function
    save_model_checkpoint(model=model, save_path=save_path)

    print("✅ Training finished, model saved as final_deeponet_model.pt")
    print("✅ Scalers saved for consistent evaluation")
    print(f"📁 Final model saved at: {save_path}")
    print(f"📁 Checkpoints saved at: {checkpoint_dir}")
    print(f"📁 Scalers saved at: {DRIVE_PATH}*_Scalar_Training.pkl")

    experiment.end()

if __name__ == "__main__":
    main()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/satyabratkumarsingh/option-portfolio-encoder-decoder/96ac953a751041d48e6b2089d00b1db8

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Fitting scalers on training data...
Fitting K and S_T scalers from training set...


Fitting Cashflow Scaler: 100%|██████████| 160/160 [00:30<00:00,  5.20it/s]


Saving scalers for evaluation consistency...
✅ Successfully saved all training scalers
📊 Scaler Statistics:
K_scaler - mean: 155.0788, std: 175.6431
S_T_scaler - mean: 313.4675, std: 138.1498
Cashflow_scaler - mean: 0.4324, std: 167.9381
[Epoch 0] Updated branch/trunk scale → 0.2400


Training Epoch 0: 100%|██████████| 160/160 [03:47<00:00,  1.42s/it]


...

Training Epoch 148: 100%|██████████| 160/160 [03:44<00:00,  1.40s/it]


Epoch 148 → Current LR: 2.955196e-05
Epoch [148/500] Train → total: 0.041746, cf: 0.022958, deriv: 0.375759 | Val → total: 0.029258, cf: 0.011049, deriv: 0.364177
[Epoch 149] Updated branch/trunk scale → 1.0000


Training Epoch 149: 100%|██████████| 160/160 [03:44<00:00,  1.40s/it]


Epoch 149 → Current LR: 3.000000e-04
Epoch [149/500] Train → total: 0.039744, cf: 0.021526, deriv: 0.364375 | Val → total: 0.026495, cf: 0.009692, deriv: 0.336055
🔄 Model checkpoint saved at epoch 150: /content/drive/MyDrive/Ucl/checkpoints/deeponet_model-epoch150.pt
Early stopping triggered at epoch 149. Best val loss: 0.026774
✅ Training finished, model saved as final_deeponet_model.pt
✅ Scalers saved for consistent evaluation
📁 Final model saved at: /content/drive/MyDrive/Ucl/final_deeponet_model.pt
📁 Checkpoints saved at: /content/drive/MyDrive/Ucl/checkpoints/
📁 Scalers saved at: /content/drive/MyDrive/Ucl/*_Scalar_Training.pkl


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : static_bug_5138
COMET INFO:     url                   : https://www.comet.com/satyabratkumarsingh/option-portfolio-encoder-decoder/96ac953a751041d48e6b2089d00b1db8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_cashflow_loss_batch [24000]   : (0.015873372554779053, 0.7178435325622559)
COMET INFO:     train_cashflow_loss_epoch [150]     : (0.021525719726923853, 0.2982598843052983)
COMET INFO:     train_derivative_loss_batch [24000] : (0.17953824996948242, 1.1214971542358398)
COMET INFO:     train_derivative_loss_epoch